## Online Markov Decision Process
### Iheb Belgacem Feb.2020

This work is based on the study of Large Scale Markov Decision Processes
with Changing Rewards made by Cardoso A.R., Wang H. and Xu.
<br>
Markov Decision Processes provide a very useful framework for Decision Making
under uncertainty.
<br>
 MDPs can be combined with Online Convex Optimization to model situations where the agent is aware of the transition probabilities between states given the actions chosen, but where the rewards are selected dynamically by an adversary.
<br>
In this notebook we apply the approach used by the authors: which consists of using  the Follow The Leader method to the dual form of the linear pro-
gramming formulation of the MDP.



In [0]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import math
import cvxpy as cp
from cvxopt import matrix, solvers

In our example we will have an n * n grid and an agent which will want to navigate this grid. 
<br>
In this context we will have n*n states and 4 actions per state.
<br>
The state vector is a zero vector with 1 only at the current state.
The actions matrix is a n² x 4 Matrix with 0 entries everywhere except for the entry (s,i) , which takes the value 1 when the agent takes action i from state s 

---



##1. Solving the MDP Problem using the dual form of the linear optimization formulation for the Bellman's equation 

In [0]:
N = 4
states = np.zeros(N*N)
states[0]=1
actions = np.zeros((N*N,4))
actions[0][2]=1

In [0]:
### the following function is used to display the current configuration of the grid
def display(states , N):
  for i in range(N): 
    for j in range(N):
      if states[N*i+j] == 1 :
        print('|  X  |',end='')
      else :
        print('|     |',end='')
    print('\n')
  return True

In [0]:
### the following function is used to calculate the transition probabilities to a state s2 if we are in state s1 and take action a
def transition_matrix(s2,s1,a,N):
  resultat = 0
  states_now = np.zeros(N*N)
  states_now[s1]=1
  actions_now = np.zeros((N*N,4))
  actions_now[s1][a] = 1
  if np.argmax(transform(states_now,actions_now))==s2:
    resultat = 1
  return resultat

In [0]:
### the following function is used to find the new state using the current state and the chosen action
def transform (states,actions):
  N = int(math.sqrt(len(states)))
  actual_state = np.argmax(states) 
  row,column = divmod(actual_state,N)
  r = row
  c= column
  if actions[actual_state][0] ==1 :
    r = row -1
  elif actions[actual_state][1] == 1:
    r = row + 1
  elif actions[actual_state][2] == 1 : 
    c = column + 1 
  elif actions[actual_state][2] == 1 : 
    c = column -1 
  if c< 0 :
    c= 0
  if c>=N:
    c = N-1
  if r<0:
    r=0
  if r>=N:
    r=N-1
  resultat = np.zeros(N*N)
  resultat[rcn(r,c,N)] = 1
  return resultat

In [0]:
### the following function is used to convert the position of the agent from the row,column form to the coordinate of the state in the state vector
def rcn(row,column,N):
  return N*row+column

In [0]:
### the following function is used to generate a random reward function
def random_reward(N):
  return np.random.choice([-1,-0.5,0,0.5,1],size = N*N)

In [0]:
#This is an example for a reward function
c2 = -1 * np.ones(N*N*4)
c2[-1] = 100
c2[-2] = 100
c2[-3] = 100
c2[-4] = 100

In [0]:
#This is the matrix corresponding to the equality constraints
nuu = 0.5
eqconst = np.zeros((N*N,N*N*4))
for i in range(N*N):
  for j in range(N*N*4):
    row,column = divmod(j,4)
    eqconst[i][j]=-nuu*transition_matrix(i,row,column,N)
  eqconst[i][4*i:4*(i+1)]=eqconst[i][4*i:4*(i+1)]+np.array([1,1,1,1])
eqconst

array([[ 0.5,  1. ,  1. , ..., -0. , -0. , -0. ],
       [-0. , -0. , -0.5, ..., -0. , -0. , -0. ],
       [-0. , -0. , -0. , ..., -0. , -0. , -0. ],
       ...,
       [-0. , -0. , -0. , ..., -0. , -0. , -0. ],
       [-0. , -0. , -0. , ..., -0. , -0. , -0. ],
       [-0. , -0. , -0. , ...,  0.5,  0.5,  0.5]])

In [0]:
### This matrix is used for the inequalty constraints : our dual variables have to be positive because they represent the nb of times
### we were in state s and took action a
G = -1 * np.identity(N*N*4)

In order to be able to use the linear programming solver of the cvxopt library, we should write our problem in the following way : 

Minimize c^t x 


---


such that G x <= h


---

Ax = b


In [0]:
c = matrix(-1*c2)
G = matrix (G)
h = matrix(np.zeros(N*N*4))
A = matrix(eqconst)
b = matrix((1/(N*N)) * np.ones(N*N))

In [0]:
sol = solvers.lp(c,G,h,A,b)

     pcost       dcost       gap    pres   dres   k/t
 0: -1.3395e+01 -1.3395e+01  8e+01  1e-16  2e+00  1e+00
 1: -3.2155e+01 -3.2088e+01  5e+01  6e-16  9e-01  7e-01
 2: -3.7791e+01 -3.7774e+01  1e+01  2e-16  2e-01  2e-01
 3: -4.1653e+01 -4.1651e+01  1e+00  9e-16  3e-02  2e-02
 4: -4.2343e+01 -4.2343e+01  7e-02  3e-16  1e-03  1e-03
 5: -4.2384e+01 -4.2384e+01  7e-04  4e-16  1e-05  1e-05
 6: -4.2385e+01 -4.2385e+01  7e-06  4e-16  1e-07  1e-07
 7: -4.2385e+01 -4.2385e+01  7e-08  2e-16  1e-09  1e-09
Optimal solution found.


In [0]:
solution = sol['x']
solution = np.array(solution)

In [0]:
### the following function is used to derive the optimal policy from the solution of the Dual LP Problem
def find_policy(solution) : 
  policy = np.zeros(N*N)
  for i in range(N*N):
    a=solution[4*i][0]
    b=solution[4*i+1][0]
    c=solution[4*i+2][0]
    d=solution[4*i+3][0]
    compare = np.array([a,b,c,d])
    policy[i] = np.argmax(compare)
  return policy

In [0]:
policy = find_policy(solution)

In [0]:
direction_dictionary = {0:'  up ',1:'down ',2:'right',3:'left '}

In [0]:
### the following function is used to display the policy we found using the LP approach
def display_solution (N, policy):
  for i in range(N): 
    for j in range(N):
        print('| '+direction_dictionary[policy[4*i+j]]+' |',end='')
    print('\n')
  return True

In [0]:
### the following function is used to display the reward function
def display_rewards(c):
  for i in range(N): 
    for j in range(N):
        print('| '+str(c[4*(N*i+j)])+' |',end='')
    print('\n')
  return True


In [0]:
display_rewards(c2)

| -1.0 || -1.0 || -1.0 || -1.0 |

| -1.0 || -1.0 || -1.0 || -1.0 |

| -1.0 || -1.0 || -1.0 || -1.0 |

| -1.0 || -1.0 || -1.0 || 100.0 |



True

In [0]:
display_solution(N,policy)

| down  || right || down  || down  |

| down  || down  || down  || down  |

| right || right || down  || down  |

| right || right || right || down  |



True

Let's try with another reward function

In [0]:
#This is an example for a reward function
c2 = np.zeros(N*N*4)
c2[7*4+0] = 50
c2[7*4+1] = 50
c2[7*4+2] = 50
c2[7*4+3] = 50

In [0]:
display_rewards(c2)

| 0.0 || 0.0 || 0.0 || 0.0 |

| 0.0 || 0.0 || 0.0 || 50.0 |

| 0.0 || 0.0 || 0.0 || 0.0 |

| 0.0 || 0.0 || 0.0 || 0.0 |



True

In [0]:
c = matrix(-1*c2)
G = matrix (G)
h = matrix(np.zeros(N*N*4))
A = matrix(eqconst)
b = matrix((1/(N*N)) * np.ones(N*N))
sol = solvers.lp(c,G,h,A,b)

     pcost       dcost       gap    pres   dres   k/t
 0: -7.6212e+00 -7.6212e+00  5e+01  1e-16  2e+00  1e+00
 1: -1.8470e+01 -1.8537e+01  3e+01  3e-16  1e+00  7e-01
 2: -2.2697e+01 -2.2720e+01  8e+00  1e-16  3e-01  2e-01
 3: -2.5658e+01 -2.5666e+01  1e+00  1e-16  6e-02  2e-02
 4: -2.6327e+01 -2.6327e+01  7e-02  4e-16  3e-03  1e-03
 5: -2.6367e+01 -2.6367e+01  7e-04  3e-16  3e-05  1e-05
 6: -2.6367e+01 -2.6367e+01  7e-06  5e-16  3e-07  1e-07
 7: -2.6367e+01 -2.6367e+01  7e-08  4e-16  3e-09  1e-09
Optimal solution found.


In [0]:
solution = sol['x']
solution = np.array(solution)

In [0]:
policy = find_policy(solution)
display_solution(N,policy)

| right || right || down  || down  |

| right || right || right || right |

| right || right ||   up  ||   up  |

| right || right ||   up  ||   up  |



True



---



## 2. Using the Follow the leader approach to deal with the case of MDP with varying reward function

The next part is about defining the random rewards for the different steps of the game a number of random rewards



In [0]:
steps  = 1000
reward_sa = np.zeros((steps,N*N*4))
for i in range(steps):
  reward_random = random_reward(N)
  for w in range(N*N):
    for q in range(4):
      reward_sa[i,4*w+q]=reward_random[action_new_state(w,q)]

In [0]:
### the following function is used to calculate the combined reward function for the previous time steps of the game
def reward_function_at_t(reward_sa,step):
  resultat= np.zeros(N*N*4)
  for i in range(steps):
    resultat = resultat + reward_sa[i,:]
  return resultat

In [0]:
### the following function is used to find the new state given the previous state and the action chosen
def action_new_state(s,a):
  i=0
  while (transition_matrix(i,s,a,N)== 0) and (i<N*N):
    i = i+1
  return i

Follow the leader ! 

In [0]:
def FTL(N,input_state,input_solution,steps,Eqconst):
  states = list()
  actions = list()
  solutions = list()
  policies = list()
  rewards = list()
  states.append(input_state)
  solutions.append(input_solution)
  for i in range(steps) :
    actions.append(int(find_policy(solutions[i])[states[i]]))
    rewards.append(reward_sa[i,N*states[i]+actions[i]])
    c = matrix(-1*reward_function_at_t(reward_sa,i))
    G = -1 * np.identity(N*N*4)
    G = matrix (G)
    h = matrix(np.zeros(N*N*4))
    A = matrix(Eqconst)
    b = matrix((1/(N*N)) * np.ones(N*N))
    sol = solvers.lp(c,G,h,A,b)
    solutions.append(np.array(sol['x']))
    states.append(action_new_state(states[-1],actions[-1]))
  return [states,actions,rewards]

In [0]:
nuu = 0.5 ## the discount factor 
eqconst = np.zeros((N*N,N*N*4)) #This is the matrix corresponding to the equality constraints
### The next lines correspond to the initialization of the algorithm
for i in range(N*N):
  for j in range(N*N*4):
    row,column = divmod(j,4)
    eqconst[i][j]=-nuu*transition_matrix(i,row,column,N)
  eqconst[i][4*i:4*(i+1)]=eqconst[i][4*i:4*(i+1)]+np.array([1,1,1,1])
input_state = int(np.random.choice(np.linspace(0,N*N-1,N*N),1)[0])
c = matrix(np.zeros(N*N*4))
G = matrix (G)
h = matrix(np.zeros(N*N*4))
A = matrix(eqconst)
b = matrix((1/(N*N)) * np.ones(N*N))
sol = solvers.lp(c,G,h,A,b)
input_solution = np.array(sol['x'])
states,actions,rewards = FTL(N,input_state,input_solution,steps,eqconst)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 0:  3.1031e+01  3.1031e+01  1e+02  1e-16  2e+00  1e+00
 1:  7.9857e+00  8.2348e+00  8e+01  2e-16  1e+00  9e-01
 2: -3.7183e+00 -3.6512e+00  2e+01  9e-17  4e-01  2e-01
 3: -8.7136e+00 -8.6457e+00  1e+01  2e-16  2e-01  2e-01
 4: -1.3926e+01 -1.3916e+01  2e+00  1e-16  3e-02  3e-02
 5: -1.5069e+01 -1.5069e+01  4e-02  3e-16  7e-04  6e-04
 6: -1.5094e+01 -1.5094e+01  4e-04  2e-16  7e-06  6e-06
 7: -1.5094e+01 -1.5094e+01  4e-06  2e-16  7e-08  6e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres   k/t
 0:  3.1031e+01  3.1031e+01  1e+02  1e-16  2e+00  1e+00
 1:  7.9857e+00  8.2348e+00  8e+01  2e-16  1e+00  9e-01
 2: -3.7183e+00 -3.6512e+00  2e+01  9e-17  4e-01  2e-01
 3: -8.7136e+00 -8.6457e+00  1e+01  2e-16  2e-01  2e-01
 4: -1.3926e+01 -1.3916e+01  2e+00  1e-16  3e-02  3e-02
 5: -1.5069e+01 -1.5069e+01  4e-02  3e-16  7e-04  6e-04
 6: -1.5094e+01 -1.5094e+01  4e-04  2e-16  7e-06  6e-06
 7: 

In [0]:
println('This is the reward for the Follow the leader approach' + str(np.mean(np.array(rewards))))

-0.003

In [0]:
### Let's compare the previous result with the case where we made the choices for the actions completely randomly
def random_method (N,input_state,steps,Eqconst):
  states = list()
  actions = list()
  rewards = list()
  states.append(input_state)
  for i in range(steps) :
    actions.append(int(np.random.choice(np.linspace(0,3,4),1)))
    rewards.append(reward_sa[i,N*states[i]+actions[i]])
    states.append(action_new_state(states[-1],actions[-1]))
  return [states,actions,rewards]

In [0]:
states, actions, rewards =random_method(N,input_state,steps,eqconst)

In [0]:
println('This is the reward for the Follow the leader approach' + str(np.mean(np.array(rewards))))


-0.012